In [1]:
%pip install ultralytics
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.227 🚀 Python-3.8.17 torch-1.12.1+cpu CPU (Intel Core(TM) i5-8265U 1.60GHz)
Setup complete ✅ (8 CPUs, 7.9 GB RAM, 199.5/221.5 GB disk)


In [1]:
import cv2
import mediapipe as mp
from ultralytics import YOLO
import numpy as np

In [2]:
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

In [10]:
import torch

if torch.cuda.is_available() == True:
    device = 'cuda:0'
else:
    device = 'cpu'

print(device)

cuda:0


In [11]:
class hand_LSTM(nn.Module):
    def __init__(self):
        super(hand_LSTM, self).__init__()
        self.lstm1 = nn.LSTM(input_size = 65, hidden_size = 128, num_layers=1, batch_first=True) # xyz 63 + x_center, y_center = 65
        self.lstm2 = nn.LSTM(input_size = 128, hidden_size = 256, num_layers=1, batch_first=True)
        self.lstm3 = nn.LSTM(input_size = 256, hidden_size = 512, num_layers=1, batch_first=True)
        self.dropout1 = nn.Dropout(0.1)
        self.lstm4 = nn.LSTM(input_size = 512, hidden_size = 256, num_layers=1, batch_first=True)
        self.lstm5 = nn.LSTM(input_size = 256, hidden_size = 128, num_layers=1, batch_first=True)
        self.lstm6 = nn.LSTM(input_size = 128, hidden_size = 64, num_layers=1, batch_first=True)
        self.dropout2 = nn.Dropout(0.1)
        self.lstm7 = nn.LSTM(input_size = 64, hidden_size = 32, num_layers=1, batch_first=True)
        self.fc = nn.Linear(32, 2)
    def forward(self, x):
        x, _ = self.lstm1(x)
        x, _ = self.lstm2(x)
        x, _ = self.lstm3(x)
        x = self.dropout1(x)
        x, _ = self.lstm4(x)
        x, _ = self.lstm5(x)
        x, _ = self.lstm6(x)
        x = self.dropout2(x)
        x, _ = self.lstm7(x)
        x = self.fc(x[:, -1,:])
        return x

In [17]:
# 모델 불러오기
model_path = './lstm_model.pth'

lstm_model = hand_LSTM().to(device)
lstm_model.load_state_dict(torch.load(model_path, map_location=device))
lstm_model.eval()
print("모델이 성공적으로 불러와졌습니다.")

모델이 성공적으로 불러와졌습니다.


In [13]:
# YOLO 객체 감지 모델 초기화
# best_model = '/content/drive/MyDrive/Deep_learning_Team_PJ/hand_tracing/yolo_eraser/best.pt'
best_model = 'best.pt'
yolo_model = YOLO(best_model)

In [7]:
from torch.utils.data import Dataset, DataLoader

class MyDataset(Dataset):
    def __init__(self, seq_list):
        self.X = []
        self.y = []
        for dic in seq_list:
            self.y.append(dic['key'])
            self.X.append(dic['value'])
    def __getitem__(self, index):
        data = self.X[index]
        label = self.y[index]
        return torch.Tensor(np.array(data)), torch.tensor(np.array(int(label)))
    
    def __len__(self):
        return len(self.X)

In [30]:
length = 20
interval = 1
detect_cls = 1

video_path = './data/video/test_data/test_video.mp4'
# cv2.destroyAllWindows()
cap = cv2.VideoCapture(video_path)
img_list = []
if cap.isOpened():
    cnt = 0
    while True:
        ret , img = cap.read()
        if ret:
            img = cv2.resize(img, (640, 640))
            if cnt == interval:
                img_list.append(img)
                cnt =0
            # cv2.imshow('test_video', img)
            # cv2.waitKey(1)
            cnt += 1

        else: break
cap.release()
# cv2.destroyAllWindows()

print('저장된 frame의 개수 {}'.format(len(img_list)))

저장된 frame의 개수 494


In [31]:
from tqdm import tqdm

lstm_model.eval()
out_img_list = []
dataset = []
status = 'None'

detect_cls = 0

mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

print('시퀀스 데이터 분석 중..')
xyz_list_list = []

for img in tqdm(img_list):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = hands.process(img)
    xyz_list = []
    filtered_boxes, filtered_box_class = [], []

    if not results.multi_hand_landmarks:
          continue
    
    for x_y_z in results.multi_hand_landmarks:
          for landmark in x_y_z.landmark:
            xyz_list.append(landmark.x)
            xyz_list.append(landmark.y)
            xyz_list.append(landmark.z)

          mp_drawing.draw_landmarks(img, x_y_z, mp_hands.HAND_CONNECTIONS)
    
    # YOLO 박스
    box_results = yolo_model.predict(img)
    box_results = box_results[0].boxes
    boxes = box_results.xyxy.cpu().tolist()
    box_class = box_results.cls.cpu().tolist()
    confs = box_results.conf.float().cpu().tolist()
    
    confidence_threshold = 0.5
    for idx, conf in enumerate(confs):
        if conf >= confidence_threshold:
            filtered_boxes.append(boxes[idx])
            filtered_box_class.append(box_class[idx])

    x_center, y_center = 0, 0
    for idx, cls in enumerate(filtered_box_class):
        if int(cls) == detect_cls or int(cls) == 1:
            x1, y1, x2, y2 = filtered_boxes[idx]
            x_center = (x1 + x2)/2
            y_center = (y1 + y2)/2
            cv2.rectangle(img, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
    
    xyz_list.append(x_center)
    xyz_list.append(y_center)

    xyz_list_list.append(xyz_list)

    if len(xyz_list_list) == length:
        dataset = []
        dataset.append({'key': 0, 'value': xyz_list_list})
        dataset = MyDataset(dataset)
        dataset = DataLoader(dataset)
        xyz_list_list = []
        for data, label in dataset:
            data = data.to(device)
            with torch.no_grad():
                result = lstm_model(data)
                _, out = torch.max(result, 1)
                if out.item() == 0: status = 'Release'
                else: status = 'Grab'

    cv2.putText(img, status, (0, 50), cv2.FONT_HERSHEY_COMPLEX, 1.5, (0,0, 225), 2)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    out_img_list.append(img)

시퀀스 데이터 분석 중..


  0%|          | 0/494 [00:00<?, ?it/s]


0: 640x640 3 red_erasers, 60.8ms
Speed: 9.0ms preprocess, 60.8ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)


  0%|          | 1/494 [00:00<01:07,  7.34it/s]


0: 640x640 3 red_erasers, 9.0ms
Speed: 4.0ms preprocess, 9.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 3 red_erasers, 9.0ms
Speed: 3.0ms preprocess, 9.0ms inference, 1.9ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 red_erasers, 10.9ms
Speed: 4.0ms preprocess, 10.9ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)


...

 99%|█████████▉| 490/494 [00:18<00:00, 27.84it/s]


0: 640x640 (no detections), 19.0ms
Speed: 3.0ms preprocess, 19.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 14.9ms
Speed: 3.0ms preprocess, 14.9ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 13.9ms
Speed: 3.0ms preprocess, 13.9ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)


100%|█████████▉| 493/494 [00:18<00:00, 26.88it/s]


0: 640x640 (no detections), 17.9ms
Speed: 3.0ms preprocess, 17.9ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)


100%|██████████| 494/494 [00:18<00:00, 26.35it/s]


In [32]:
filename = './video_out.mp4'
fourcc = cv2.VideoWriter_fourcc(*'DIVX')

fps = 30
frameSize = (640,640)
isColor = True
out = cv2.VideoWriter(filename, fourcc, fps, frameSize, isColor)
for out_img in out_img_list:
    out.write(out_img)

out.release()